In [8]:
import pandas as pd
import flavor


/Users/vmasrani/miniconda/envs/ml3/lib/python3.10/site-packages/polars/meta/build.py:5: UserWarning: Polars binary is missing!
  from polars.utils._polars_version import get_polars_version


In [9]:
df = pd.read_csv('sample_data.csv')
df

,date,value,category,id
0,01-01-2023,100.727824,RED,0
1,2023-01-02,87.535748,red,1
2,03-01-2023,104.056852,blue,2
3,04-01-2023,99.246428,blue,3
4,2023-01-05,96.415779,BLUE,4
...,...,...,...,...
1045,09/26/2024,97.749166,GREEN,634
1046,01/23/2023,109.578887,BLUE,22
1047,10/17/2024,75.319019,GREEN,655
1048,2024-06-04,109.202770,GREEN,520


In [3]:
import pandas as pd
from pathlib import Path
from openai import OpenAI
import json
from pydantic import BaseModel
from dotenv import load_dotenv
completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {
                "role": "system",
                "content": """Map CSV headers to standardized clean headers following these rules:
                    1. Maintain one-to-one mapping between dirty and clean headers
                    2. NO DUPLICATE clean headers allowed
                    3. Preserve information when combining fields
                       Example: 'job_company_location_street_address' -> 'company_street_address'
                    4. Create new headers if no valid match exists
                    5. Use CSV row context for better mapping decisions
                    6. Preserve original header if already clean
                    7. Keep 'zip' and 'postal_code' distinct
                    8. All headers m>ust contain a 'email' field (or 'email' and 'work_email' if two email fields are present)
                    9. map urls to 'source'
                    Return headers in original order."""
            },
            {
                "role": "user",
                "content": f"""CSV preview (header may be in first row):
                    {first_five}
                    Map to these valid headers:
                    {valid_headers}
                    Return ordered lists of dirty and clean headers."""
            }
        ],
        temperature=0.0,
        response_format=CleanHeaders
    )


In [5]:
completion

ParsedChatCompletion[NoneType](id='chatcmpl-AxN2HCkptDGKdTWxx44NF8QhCycY0', choices=[ParsedChoice[NoneType](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[NoneType](content='The Moon does not have a capital. It is a natural satellite of Earth and not a sovereign entity or inhabited territory with its own governance system. There have been human missions and landings on the Moon, such as the Apollo missions conducted by NASA, but no permanent settlements or capitals have been established.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[], parsed=None))], created=1738713265, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_50cad350e4', usage=CompletionUsage(completion_tokens=61, prompt_tokens=32, total_tokens=93, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_de

In [4]:
completions

In [ ]:


import json
from ml_helpers import chunks
import flavor

load_dotenv()
client = OpenAI()

class CleanHeaders(BaseModel):
    dirty_headers: list[str]
    clean_headers: list[str]

def get_first_n_rows(path: Path, nrows: int = 5) -> pd.DataFrame:
    return pd.read_csv(
        path,
        encoding="utf-8",
        encoding_errors='replace',
        nrows=nrows
    )

# 4. Pay special attention to social media fields. Don't lose social media information.
def openai_header_clean(first_five, valid_headers):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {
                "role": "system",
                "content": """Map CSV headers to standardized clean headers following these rules:
                    1. Maintain one-to-one mapping between dirty and clean headers
                    2. NO DUPLICATE clean headers allowed
                    3. Preserve information when combining fields
                       Example: 'job_company_location_street_address' -> 'company_street_address'
                    4. Create new headers if no valid match exists
                    5. Use CSV row context for better mapping decisions
                    6. Preserve original header if already clean
                    7. Keep 'zip' and 'postal_code' distinct
                    8. All headers m>ust contain a 'email' field (or 'email' and 'work_email' if two email fields are present)
                    9. map urls to 'source'
                    Return headers in original order."""
            },
            {
                "role": "user",
                "content": f"""CSV preview (header may be in first row):
                    {first_five}

                    Map to these valid headers:
                    {valid_headers}

                    Return ordered lists of dirty and clean headers."""
            }
        ],
        temperature=0.0,
        response_format=CleanHeaders
    )

    message = completion.choices[0].message
    if message.parsed:
        return pd.DataFrame({
            'dirty_headers': message.parsed.dirty_headers,
            'clean_headers': message.parsed.clean_headers,
        })
    else:
        print(message.refusal)